This kernel is a copy of a [kernel](https://www.kaggle.com/kaushal2896/bengali-graphemes-starter-eda-multi-output-cnn). But with a lot of comments. It is used only for educational purposes.

**Загрузка библиотек и начальный анализ входных данных**

In [ ]:
## Блок частоупотребимых библиотек по всем темам анализа данных
# линейная алгебра https://pythonworld.ru/numpy/1.html  https://habr.com/ru/post/352678/  https://python-scripts.com/numpy
import numpy as np
# чтение файлов и обработка данных https://proglib.io/p/pandas-tricks   https://khashtamov.com/ru/pandas-introduction/ https://habr.com/ru/company/ods/blog/322626/
import pandas as pd 
# sklearn - препроцессинг, метрики моделей, валидация моделей, выбор признако, выбор моделей,
# модели машинного обучения: классификация, регрессия, кластеризция, понижение размерности...
# Это основная библиотека для классического машинного обучения https://scikit-learn.org/stable/     https://habr.com/ru/company/mlclass/blog/247751/
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
# Визуализация https://python-scripts.com/matplotlib  https://pythonworld.ru/novosti-mira-python/scientific-graphics-in-python.html
import matplotlib.image as mpimg
from matplotlib import pyplot as plt
# https://habr.com/ru/company/ods/blog/323210/  https://nagornyy.me/courses/data-science/intro-to-seaborn/
import seaborn as sns # более высокоуровневая и легкая в использовании
## конец блока частоупотребимых библиотек

## Блок библиотек связанных с задачами компьютерного зрения
# Алгоритмы обработки изображений https://tproger.ru/translations/opencv-python-guide/  https://arboook.com/kompyuternoe-zrenie/osnovnye-operatsii-s-izobrazheniyami-v-opencv-3-python/
import cv2
# https://habr.com/ru/post/451074/   https://pythonru.com/biblioteki/osnovnye-vozmozhnosti-biblioteki-python-imaging-library-pillow-pil
import PIL.Image as Image, PIL.ImageDraw as ImageDraw, PIL.ImageFont as ImageFont
## конец блока библиотек для работы с изображениями

## Блок работы с нейронными сетями
# keras - высокоуровневая библиотека для работы с нейронными сетями. Работает как надстройка над tensorflow или theano (на выбор)
# видео лекции от Созыкина А.В. https://www.youtube.com/watch?v=GX7qxV5nh5o&list=PLtPJ9lKvJ4oiz9aaL_xcZd-x0qd8G0VN_
# https://www.youtube.com/watch?v=52U4BG0ENiM&list=PLtPJ9lKvJ4oi5ATzKmmp6FznCHmnhVoey
# туториал https://riptutorial.com/ru/keras/topic/8695/%D0%BD%D0%B0%D1%87%D0%B0%D0%BB%D0%BE-%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D1%8B-%D1%81-%D0%BA%D0%B5%D1%80%D0%B0%D1%81%D0%BE%D0%BC
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# крутой и краткий туториал на habr https://habr.com/ru/company/ods/blog/325432/
from tensorflow.keras.models import Model
from tensorflow.keras.models import clone_model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPool2D, Dropout, BatchNormalization, Input, GlobalAveragePooling2D
# Оптимизаторы https://habr.com/ru/post/318970/
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
# предварительно обученные нейронные сети
# Архитектуры в Керас https://www.pyimagesearch.com/2017/03/20/imagenet-vggnet-resnet-inception-xception-keras/
# Архитектуры своими руками https://towardsdatascience.com/cnn-architectures-a-deep-dive-a99441d18049
# tensorflow docs https://www.tensorflow.org/api_docs/python/tf/keras/applications
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.densenet import DenseNet121, DenseNet169, DenseNet201 
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet_v2 import ResNet50V2, ResNet101V2, ResNet152V2
# пример нейронки с несколькими выходами https://www.pyimagesearch.com/2018/06/04/keras-multiple-outputs-and-multiple-losses/
## конец блока нейронок

## Блок вспомогательных библиотек
# визуализация прогресса https://stackoverflow.com/questions/42212810/tqdm-in-jupyter-notebook-prints-new-progress-bars-repeatedly
from tqdm.auto import tqdm 
# позволяет выбирать список файлов по шаблону пути https://pythonworld.ru/moduli/modul-glob.html
from glob import glob 
# время и сборщик мусора https://all-python.ru/osnovy/modul-time.html  https://asvetlov.blogspot.com/2013/05/gc.html http://www.ilnurgi1.ru/docs/python/modules/gc.html
import time, gc
## конец блока вспомогательных библиотек

Просмотр входных данных

In [ ]:
from pathlib import Path
featherdir = Path('/kaggle/input/bengaliaicv19feather') # папка с изображениями в формате feather
import os
#выведем все файлы в директории "input/"
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Любые результаты, которые вы записываете в текущий каталог, сохраняются как выходные данные.

Сравним скорость работы с файлами "feather" и "parquet"

In [ ]:
# %%time
# train_image_df0 = pd.read_feather(featherdir/'train_image_data_0.feather')
# train_image_df1 = pd.read_feather(featherdir/'train_image_data_1.feather')
# train_image_df2 = pd.read_feather(featherdir/'train_image_data_2.feather')
# train_image_df3 = pd.read_feather(featherdir/'train_image_data_3.feather')
# train_image_df3.head()

In [ ]:
# %%time
# d = pd.read_parquet(f'/kaggle/input/bengaliai-cv19/train_image_data_1.parquet')
# d.head()

In [ ]:
# del train_image_df0
# del train_image_df1
# del train_image_df2
# del train_image_df3
# del d
# gc.collect()


In [ ]:
# данные для тренировки
train_df_ = pd.read_csv('/kaggle/input/bengaliai-cv19/train.csv')
# данные для тестирования алгоритма
test_df_ = pd.read_csv('/kaggle/input/bengaliai-cv19/test.csv')
# данные для нахождения составных частей графемы (корня графемы, гласного и согласного диакретических знаков)
class_map_df = pd.read_csv('/kaggle/input/bengaliai-cv19/class_map.csv')
# пример оформления файла ответов
sample_sub_df = pd.read_csv('/kaggle/input/bengaliai-cv19/sample_submission.csv')

In [ ]:
train_df_.head() # смотрим какие данные в тренировочной таблице

In [ ]:
#  смотрим количество уникальных значений в каждом столбце
train_df_.grapheme_root.nunique(), train_df_.vowel_diacritic.nunique(), train_df_.consonant_diacritic.nunique(), train_df_.grapheme.nunique()

In [ ]:
test_df_.head()  # смотрим какие данные в тестовой таблице

In [ ]:
sample_sub_df.head()  # смотрим какие данные в примере файла ответов

In [ ]:
# таблица, которая ставит в соответствие некой полной графемы G ее состовляющие:
# корень графемы, гласные диакритические знаки и согласные диакритические знаки
G = 5
class_map_df[class_map_df.label==G]

In [ ]:
class_map_df.head()

In [ ]:
class_map_df.component_type.nunique(), class_map_df.label.nunique(), class_map_df.component.nunique()

In [ ]:
 # смотрим размеры наборов данных
print(f'Size of training data: {train_df_.shape}')
print(f'Size of test data: {test_df_.shape}')
print(f'Size of class map: {class_map_df.shape}')

## Exploratory Data Analysis
Исследовательский анализ данных (EDA) - это подход к анализу наборов данных для обобщения их основных характеристик, часто с помощью визуальных методов.

In [ ]:
HEIGHT = 236 # высота изображения
WIDTH = 236 # ширина изображения
# если хотите глубже понять, что происходит - раскоментируйте принты
def get_n(df, field, n, top=True):
    '''функция для выбора топ N графем желаемого типа (field)
        df - data frame с графемами (в нашем случае test_df_)
        field - часть графемы, которую хотим проанализировать. Может принимать 3 значения (grapheme_root, vowel_diacritic, consonant_diacritic)
        n - число знаков, которые мы хотим увидеть
        top - (True/False) признак сортировки (по убывание/ по возрастанию)
    '''
    # группируем датасет по выбранной части графемы и считаем количество появлений каждого типа графемы. Сортируем, отсекам топ
    top_graphemes = df.groupby([field]).size().reset_index(name='counts')['counts'].sort_values(ascending=not top)[:n]
#     print(top_graphemes)
    top_grapheme_roots = top_graphemes.index # отдельно выбираем индексы (номера частей графем)
#     print(top_grapheme_roots)
    top_grapheme_counts = top_graphemes.values # отдельно выбираем значение (количества) вхождения этой части графемы в набор данных
#     print(top_grapheme_counts)
    # находим выбранные нами части графем в наборе с рисунками этиъ частей графем
    top_graphemes = class_map_df[class_map_df['component_type'] == field].reset_index().iloc[top_grapheme_roots]
#     print(top_graphemes)
    top_graphemes.drop(['component_type', 'label'], axis=1, inplace=True) # удаляем ненужные нам столбцы
#     print(top_graphemes)
    top_graphemes.loc[:, 'count'] = top_grapheme_counts # добавляем к номерам и рисункам частей графем количества их вхождения
#     print(top_graphemes)
    return top_graphemes

def image_from_char(char):
    '''функция отображения символов из таблицы с данными в рисунки желаемого размера
        char - часть графемы, которую хотим отрисовать в увеличенном масштабе
    '''
    image = Image.new('RGB', (WIDTH, HEIGHT)) # задаем трехканальный тип изображения и его размерность
    draw = ImageDraw.Draw(image) # создаем объект для отрисовки
    myfont = ImageFont.truetype('/kaggle/input/kalpurush-fonts/kalpurush-2.ttf', 120) # выбираем тип шрифта и его размер
    w, h = draw.textsize(char, font=myfont) # преобразовуем табличную графему к выбранному шрифту и размерам шрифта
    draw.text(((WIDTH - w) / 2,(HEIGHT - h) / 3), char, font=myfont) # рисуем символ, отцентровав его на полотне изображения

    return image

* ### Число уникальных значений по типам графем, которые требуется предсказать

In [ ]:
print(f'Number of unique grapheme roots: {train_df_["grapheme_root"].nunique()}')
print(f'Number of unique vowel diacritic: {train_df_["vowel_diacritic"].nunique()}')
print(f'Number of unique consonant diacritic: {train_df_["consonant_diacritic"].nunique()}')

### Топ 10 наиболее используемых корней графем

In [ ]:
top_10_roots = get_n(train_df_, 'grapheme_root', 10)
top_10_roots

In [ ]:
# создаем сетку 2 на 5, для более компактного отображения символов и задаем размер их отображения
f, ax = plt.subplots(2, 5, figsize=(16, 8))
ax = ax.flatten()
# отрисовываем в цикле найденные топ N изображений частей графем
for i in range(10):
    ax[i].imshow(image_from_char(top_10_roots['component'].iloc[i]), cmap='Greys')

* ### Топ 10 наименее используемых корней графем в наборе

In [ ]:
bottom_10_roots = get_n(train_df_, 'grapheme_root', 10, False)
bottom_10_roots

In [ ]:
f, ax = plt.subplots(2, 5, figsize=(16, 8))
ax = ax.flatten()

for i in range(10):
    ax[i].imshow(image_from_char(bottom_10_roots['component'].iloc[i]), cmap='Greys')

* ### Топ 5 гласных диакритических знаков в тренировочном наборе данных

In [ ]:
top_5_vowels = get_n(train_df_, 'vowel_diacritic', 5)
top_5_vowels

In [ ]:
f, ax = plt.subplots(1, 5, figsize=(16, 8))
ax = ax.flatten()

for i in range(5):
    ax[i].imshow(image_from_char(top_5_vowels['component'].iloc[i]), cmap='Greys')

* ### Топ 5 согласных диакритических знаков в тренировочном наборе данных

In [ ]:
top_5_consonants = get_n(train_df_, 'consonant_diacritic', 5)
top_5_consonants

In [ ]:
f, ax = plt.subplots(1, 5, figsize=(16, 8))
ax = ax.flatten()

for i in range(5):
    ax[i].imshow(image_from_char(top_5_consonants['component'].iloc[i]), cmap='Greys')

In [ ]:
train_df_ = train_df_.drop(['grapheme'], axis=1, inplace=False) # убираем из тренировочного набора данных рисунки графем

In [ ]:
# преобразовуем тип данных к типу, который занимает меньше места в памяти 
train_df_[['grapheme_root', 'vowel_diacritic', 'consonant_diacritic']] = train_df_[['grapheme_root', 'vowel_diacritic', 'consonant_diacritic']].astype('uint8')

In [ ]:
IMG_SIZE=64 # определяем входной размер изображений для нейронной сети
N_CHANNELS=1 # определяем число каналов цвета для нейронной сети

Применим некоторые преобразования над изображениями (credits: [this kernel](https://www.kaggle.com/shawon10/bangla-graphemes-image-processing-deep-cnn)). Изменим их размер и отцентруем графемы.

In [ ]:
def resize(df, size=64, need_progress_bar=True):
    '''функция преобразовывает изображения к квадратной форме необходимого размера
        df - набор данных с изображениями исходного размера
        size - ширина и высота изображения после преобразования
        need_progress_bar - вывод строки состояния
        return - дата фрейм содержащий изображения нового размера
    '''
    resized = {}
    resize_size=size
    # если нужно выводить строку состояния, то оборачиваем цикл в tqdm, иначе нет
    if need_progress_bar:
        for i in tqdm(range(df.shape[0])):
            # извлекаем строку с одним изображением и преобразуем вектор в матрицу изображения исходного размера
            image=df.loc[df.index[i]].values.reshape(137,236)
            # приводим изображение к бинарному
            _, thresh = cv2.threshold(image, 30, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
            # применяем алгоритм поиска всех вероятных контуров изображения
            contours, _ = cv2.findContours(thresh,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[-2:]

            ls_xmin = []
            ls_ymin = []
            ls_xmax = []
            ls_ymax = []
            # для всех найденных контуров
            for cnt in contours:
                # находим координаты прямоугольника описанного вокруг найденного контура изображения
                x,y,w,h = cv2.boundingRect(cnt)
                # помещаем координаты углов прямоугольника, каждый в свой массив
                ls_xmin.append(x)
                ls_ymin.append(y)
                ls_xmax.append(x + w)
                ls_ymax.append(y + h)
            #  после перебора всех возможных контуров, находим крайние относительно центра координаты прямоугольника, чтобы наверняка захватить все изображение
            xmin = min(ls_xmin)
            ymin = min(ls_ymin)
            xmax = max(ls_xmax)
            ymax = max(ls_ymax)
            # обрезаем изображение по полученным координатам (теперь у нас есть изображение графемы, занимающее максимум объема изображения)
            roi = image[ymin:ymax,xmin:xmax]
            # меняем размер изображения на необходимый нам
            resized_roi = cv2.resize(roi, (resize_size, resize_size),interpolation=cv2.INTER_AREA)
            # добавляем новое изображение в словарь
            resized[df.index[i]] = resized_roi.reshape(-1)
    else:
        for i in range(df.shape[0]):
            image=df.loc[df.index[i]].values.reshape(137,236)
            _, thresh = cv2.threshold(image, 30, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
            contours, _ = cv2.findContours(thresh,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[-2:]

            ls_xmin = []
            ls_ymin = []
            ls_xmax = []
            ls_ymax = []
            for cnt in contours:
                x,y,w,h = cv2.boundingRect(cnt)
                ls_xmin.append(x)
                ls_ymin.append(y)
                ls_xmax.append(x + w)
                ls_ymax.append(y + h)
            xmin = min(ls_xmin)
            ymin = min(ls_ymin)
            xmax = max(ls_xmax)
            ymax = max(ls_ymax)

            roi = image[ymin:ymax,xmin:xmax]
            resized_roi = cv2.resize(roi, (resize_size, resize_size),interpolation=cv2.INTER_AREA)
            resized[df.index[i]] = resized_roi.reshape(-1)
    # создаем из полученного словаря с изображениями нового размера дата фрейм и возвращаем его
    resized = pd.DataFrame(resized).T
    return resized

In [ ]:
def get_dummies(df):
    cols = []
    for col in df:
        cols.append(pd.get_dummies(df[col].astype(str)))
    return pd.concat(cols, axis=1)

In [ ]:
d = pd.DataFrame({1:[8,1,3,4,5,6,],2:[0,15,54,0,4,8,],3:[10,11,45,0,7,9,],4:[0,1,3,4,5,6,]}).T
print(d)
cols = []
for col in d:
#     print(col)
#     print(pd.get_dummies(d[col].astype(str)))
    cols.append(pd.get_dummies(d[col].astype(str)))
pd.concat(cols, axis=1)

## Базовая модель

In [ ]:
# Создаем базовую модель блока свертки с использованием Keras
def make_conv_block(input_tensor, num_filters):
    '''функция создает блок из сверточных слоев, слоя пакетной нормализации (BatchNormalization), слоя MaxPool2D и Dropout
            df - набор данных с изображениями исходного размера
            size - ширина и высота изображения после преобразования
            need_progress_bar - вывод строки состояния
    return - дата фрейм содержащий изображения нового размера'''
    # серия сверточных слоев с одинаковым размером kernel_size
    model = Conv2D(filters=num_filters, kernel_size=(3, 3), padding='SAME', activation='relu')(input_tensor)
    model = Conv2D(filters=num_filters, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
    model = Conv2D(filters=num_filters, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
    model = Conv2D(filters=num_filters, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
    model = BatchNormalization(momentum=0.15)(model) # слой мини-пакетной нормализации
    model = MaxPool2D(pool_size=(2, 2))(model) # слой уменьшения размерности (в 2 раза)
    model = Conv2D(filters=num_filters, kernel_size=(5, 5), padding='SAME', activation='relu')(model) # еще одна свертка с ядром большего размера (5,5)
    model = Dropout(rate=0.3)(model) # слой регуляризации (случайно "замораживает" часть нейронов в слое, чтобы избежать переобучения)
    return model

# создаем серию однотипных блоков с разным количеством фильтров
inputs = Input(shape = (IMG_SIZE, IMG_SIZE, N_CHANNELS)) # входной слой 
model = inputs # небольшой трюк, связанный с тем, что inputs нам понадобиться и дальше, поэтому его нельзя переопределять и преобразовывать
# последовательно применяем созданный блок сверток со все нарастающей глубиной фильтров
for num_filters in [32, 64, 128, 256]:
    conv_block = make_conv_block(model, num_filters)
    model = conv_block
# преобразовываем выход последнего сверточного блока в плоский вектор
model = Flatten()(model)
model = Dense(1024, activation = "relu")(model) # добавляем полносвязный слой нейронной сети
model = Dropout(rate=0.3)(model) #  слой регуляризации
dense = Dense(512, activation = "relu")(model) # еще один полносвязный слой
head_root = Dense(168, activation = 'softmax')(dense) # выход нейронной сети отвечающий за классификацию графем
head_vowel = Dense(11, activation = 'softmax')(dense) # выход нейронной сети отвечающий за классификацию гласных диакректических знаков
head_consonant = Dense(7, activation = 'softmax')(dense) # выход нейронной сети отвечающий за классификацию согласных диакректических знаков
# создаем модель
model = Model(inputs=inputs, outputs=[head_root, head_vowel, head_consonant])

In [ ]:
## если хотите использовать предварительно обученную нейронную сеть
## на данный момент необходимо еще добавить преобразование изображений к трехканальным, чтобы данная модель работала
# vgg19_net = VGG19(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
# vgg19_net.trainable = False
# model = vgg19_net.output
# model = Flatten()(model)
# model = Dense(1024, activation = "relu")(model) # добавляем полносвязный слой нейронной сети
# model = Dropout(rate=0.3)(model) #  слой регуляризации
# dense = Dense(512, activation = "relu")(model) # еще один полносвязный слой
# head_root = Dense(168, activation = 'softmax')(dense) # выход нейронной сети отвечающий за классификацию графем
# head_vowel = Dense(11, activation = 'softmax')(dense) # выход нейронной сети отвечающий за классификацию гласных диакректических знаков
# head_consonant = Dense(7, activation = 'softmax')(dense) # выход нейронной сети отвечающий за классификацию согласных диакректических знаков
# создаем модель
# model = Model(inputs=vgg19_net.input, outputs=[head_root, head_vowel, head_consonant])

In [ ]:
model.summary() # выводим описание модели

Визуализируем CNN с 3 выходами

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='model.png')

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) # компилируем модель

In [ ]:
# Установим скорость обучения для каждого выхода нейронной сети.
# Так же зададим правило, что, если 3 (patience=3) эпохи подряд точность не возрастает, то делим скорость обучения пополам (factor=0.5)
learning_rate_reduction_root = ReduceLROnPlateau(monitor='dense_2_accuracy', 
                                            patience=3, 
                                            verbose=1,
                                            factor=0.5, 
                                            min_lr=0.00001)
learning_rate_reduction_vowel = ReduceLROnPlateau(monitor='dense_3_accuracy', 
                                            patience=3, 
                                            verbose=1,
                                            factor=0.5, 
                                            min_lr=0.00001)
learning_rate_reduction_consonant = ReduceLROnPlateau(monitor='dense_4_accuracy', 
                                            patience=3, 
                                            verbose=1,
                                            factor=0.5, 
                                            min_lr=0.00001)

In [ ]:
batch_size = 256
epochs = 30

In [ ]:
class MultiOutputDataGenerator(keras.preprocessing.image.ImageDataGenerator):
    '''класс наследующий класс ImageDataGenerator Keras
    '''
    def flow(self,
             x,
             y=None,
             batch_size=32,
             shuffle=True,
             sample_weight=None,
             seed=None,
             save_to_dir=None,
             save_prefix='',
             save_format='png',
             subset=None):
        '''переопределяет функцию flow родительского класса'''
        targets = None
        target_lengths = {}
        ordered_outputs = []
        for output, target in y.items():
            if targets is None:
                targets = target
            else:
                targets = np.concatenate((targets, target), axis=1)
            target_lengths[output] = target.shape[1]
            ordered_outputs.append(output)


        for flowx, flowy in super().flow(x, targets, batch_size=batch_size,
                                         shuffle=shuffle):
            target_dict = {}
            i = 0
            for output in ordered_outputs:
                target_length = target_lengths[output]
                target_dict[output] = flowy[:, i: i + target_length]
                i += target_length

            yield flowx, target_dict

In [ ]:
HEIGHT = 137
WIDTH = 236

### Training loop

In [ ]:
histories = []
# считываем по очереди учебные наборы изображений и делаем джоин train_df_ с этими изображениями, чтобы определить их метки
for i in range(4):
    train_df = pd.merge(pd.read_feather(featherdir/f'train_image_data_{i}.feather'), train_df_, on='image_id').drop(['image_id'], axis=1)
    
    # Визуализируйте несколько образцов текущего учебного набора данных
    fig, ax = plt.subplots(nrows=3, ncols=4, figsize=(16, 8))
    count=0
    for row in ax:
        for col in row:
            col.imshow(resize(train_df.drop(['grapheme_root', 'vowel_diacritic', 'consonant_diacritic'], axis=1).iloc[[count]], size=IMG_SIZE,need_progress_bar=False).values.reshape(-1).reshape(IMG_SIZE, IMG_SIZE).astype(np.float64))
            count += 1
    plt.show()
    
    # удаляем из тренировочных данных метки классов
    X_train = train_df.drop(['grapheme_root', 'vowel_diacritic', 'consonant_diacritic'], axis=1)
    X_train = resize(X_train, size=IMG_SIZE)/255 # преобразовываем изображение к размеру, который ожидает на вход нейронная сеть и нормализуем размеры пикселей
    
    # CNN принимает изображения в форме `(batch_size, h, w, channel)`, поэтому изменяйте форму изображений
    X_train = X_train.values.reshape(-1, IMG_SIZE, IMG_SIZE, N_CHANNELS)
    # преобразовываем метки классов из целочисленных значений к векторам. 
    # К примеру согласные занки: вектор-столбец Y_train_consonant содержит значения от 0 до 6, а теперь он станет матрицей из 7 столбцов такой же длины
    # И если i-тое значение было 5, то теперь это будет вектор строка [0, 0, 0, 0, 0, 1, 0]
    Y_train_root = pd.get_dummies(train_df['grapheme_root']).values
    Y_train_vowel = pd.get_dummies(train_df['vowel_diacritic']).values
    Y_train_consonant = pd.get_dummies(train_df['consonant_diacritic']).values

    print(f'Training images: {X_train.shape}')
    print(f'Training labels root: {Y_train_root.shape}')
    print(f'Training labels vowel: {Y_train_vowel.shape}')
    print(f'Training labels consonants: {Y_train_consonant.shape}')

    # Разделите данные на набор для обучения и проверки
    x_train, x_test, y_train_root, y_test_root, y_train_vowel, y_test_vowel, y_train_consonant, y_test_consonant = train_test_split(X_train, Y_train_root, Y_train_vowel, Y_train_consonant, test_size=0.08, random_state=666)
    del train_df
    del X_train
    del Y_train_root, Y_train_vowel, Y_train_consonant

    # Увеличение данных для создания большего количества обучающих данных
    datagen = MultiOutputDataGenerator(
        featurewise_center=False,  # Смещение центра - установим значение 0, для всего набора данных
        samplewise_center=False,  # Смещение центра -  установим значение 0, для каждой выборки
        featurewise_std_normalization=False,  # уберем нормализацию для всего набора данных
        samplewise_std_normalization=False,  # уберем нормализацию для каждой выборки
        zca_whitening=False,  
        rotation_range=12,  # случайным образом поворачивать изображения в диапазоне (градусы, 0 to 180)  (8)
        zoom_range = 0.20, # Случайное увеличение изображения (0.15)
        width_shift_range=0.15,  # случайное смещение изображений по горизонтали (доля от общей ширины)
        height_shift_range=0.15,  # произвольно сдвигать изображения по вертикали (доля от общей высоты)
        horizontal_flip=False,  # случайно перевернуть изображения относительно горизонтали
        vertical_flip=False)  # случайно перевернуть изображения относительно вертикали


    # Вычислим параметры, необходимые для дополнения данных. Но пока не будем выполнять какие-либо дополнения
    datagen.fit(x_train)

    # Обучим модель
    history = model.fit_generator(datagen.flow(x_train, {'dense_2': y_train_root, 'dense_3': y_train_vowel, 'dense_4': y_train_consonant}, batch_size=batch_size),
                              epochs = epochs, validation_data = (x_test, [y_test_root, y_test_vowel, y_test_consonant]), 
                              steps_per_epoch=x_train.shape[0] // batch_size, 
                              callbacks=[learning_rate_reduction_root, learning_rate_reduction_vowel, learning_rate_reduction_consonant])

    histories.append(history)
    
    # Delete to reduce memory usage
    del x_train
    del x_test
    del y_train_root
    del y_test_root
    del y_train_vowel
    del y_test_vowel
    del y_train_consonant
    del y_test_consonant
    gc.collect()

In [ ]:
# если необходимо, сохраните модель для дальнейшего использования
name_model = 'own_model_1.h5'
model.save(name_model)
# model = load_model(name_model) # загрузить готовую модель для дальнейшего использования

**Строим графики потерь и точности**

In [ ]:
%matplotlib inline
def plot_loss(his, epoch, title):
    plt.style.use('ggplot')
    plt.figure()
    plt.plot(np.arange(0, epoch), his.history['loss'], label='train_loss')
    plt.plot(np.arange(0, epoch), his.history['dense_3_loss'], label='train_root_loss')
    plt.plot(np.arange(0, epoch), his.history['dense_4_loss'], label='train_vowel_loss')
    plt.plot(np.arange(0, epoch), his.history['dense_5_loss'], label='train_consonant_loss')
    
    plt.plot(np.arange(0, epoch), his.history['val_dense_3_loss'], label='val_train_root_loss')
    plt.plot(np.arange(0, epoch), his.history['val_dense_4_loss'], label='val_train_vowel_loss')
    plt.plot(np.arange(0, epoch), his.history['val_dense_5_loss'], label='val_train_consonant_loss')
    
    plt.title(title)
    plt.xlabel('Epoch #')
    plt.ylabel('Loss')
    plt.legend(loc='upper right')
    plt.show()

def plot_acc(his, epoch, title):
    plt.style.use('ggplot')
    plt.figure()
    plt.plot(np.arange(0, epoch), his.history['dense_3_accuracy'], label='train_root_acc')
    plt.plot(np.arange(0, epoch), his.history['dense_4_accuracy'], label='train_vowel_accuracy')
    plt.plot(np.arange(0, epoch), his.history['dense_5_accuracy'], label='train_consonant_accuracy')
    
    plt.plot(np.arange(0, epoch), his.history['val_dense_3_accuracy'], label='val_root_acc')
    plt.plot(np.arange(0, epoch), his.history['val_dense_4_accuracy'], label='val_vowel_accuracy')
    plt.plot(np.arange(0, epoch), his.history['val_dense_5_accuracy'], label='val_consonant_accuracy')
    plt.title(title)
    plt.xlabel('Epoch #')
    plt.ylabel('Accuracy')
    plt.legend(loc='upper right')
    plt.show()

In [ ]:
for dataset in range(4):
    plot_loss(histories[dataset], epochs, f'Training Dataset: {dataset}')
    plot_acc(histories[dataset], epochs, f'Training Dataset: {dataset}')

In [ ]:
del histories
gc.collect()

In [ ]:
preds_dict = {
    'grapheme_root': [],
    'vowel_diacritic': [],
    'consonant_diacritic': []
}

In [ ]:
components = ['consonant_diacritic', 'grapheme_root', 'vowel_diacritic']
target=[] # список предсказаний модели
row_id=[] # список id меток предсказаний
for i in range(4):
#     df_test_img = pd.read_parquet(f'/kaggle/input/bengaliai-cv19/test_image_data_{i}.parquet') # читаем тестовый набор
    df_test_img = pd.read_feather(featherdir/f'test_image_data_{i}.feather')  # читаем тестовый набор
    df_test_img.set_index('image_id', inplace=True) # устанавливаем столбец с номером изображения, как индекс

    X_test = resize(df_test_img, size=IMG_SIZE,need_progress_bar=False)/255 # меняем размер изображения и нормализуем его
    X_test = X_test.values.reshape(-1, IMG_SIZE, IMG_SIZE, N_CHANNELS) # меняем размерность маьрицы изображения с учетом измерения для мини-пакетов
    
    preds = model.predict(X_test) # выполняем предсказание
    
    # перебираем три выхода нейронной сети
    for i, p in enumerate(preds_dict):
        preds_dict[p] = np.argmax(preds[i], axis=1) # находим вектор наиболее вероятных предсказаний для текущего типа классификатора (например для корней графем)
    # записываем предсказания в формате, который требуется для сабмита
    for k,id in enumerate(df_test_img.index.values):  
        for i,comp in enumerate(components):
            id_sample=id+'_'+comp
            row_id.append(id_sample)
            target.append(preds_dict[comp][k])
    del df_test_img
    del X_test
    gc.collect()
# преобразовываем полученные метки и предсказания в dataframe
df_sample = pd.DataFrame(
    {
        'row_id': row_id,
        'target':target
    },
    columns = ['row_id','target'] 
)
# записываем файл сабмита
df_sample.to_csv('submission.csv',index=False)
df_sample.head()